<a href="https://colab.research.google.com/github/mthsansu/MLNLP/blob/main/Code/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **In all your experiments you should compare the performance of your models with simpler models (baselines). Baseline models should correspond to the most simple model you can build to solve your task. It could be a rule-based system, or a simple machine learning / deep-learning based system**

In [1]:
# Import modules and frameworks
#! pip install transformers
#! pip install torchinfo

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import nltk
from gensim.models.phrases import Phrases, Phraser
from nltk.tokenize import TreebankWordTokenizer, TweetTokenizer
import pandas as pd
from termcolor import colored
from collections import Counter
from nltk.tokenize import TweetTokenizer
from tqdm.notebook import tqdm

In [ ]:
# Import data
from google.colab import files
files.upload()

Saving df_label_sentiment.csv to df_label_sentiment (1).csv


{'df_label_sentiment.csv': b',X,text,possibly_sensitive,created_at,twitter.x,sexe,Label,sentiment,test\r\n0,248552,"Article de Var Matin, ce jour, sur la proposition de loi que je porte pour valoriser le volontariat des sapeurs-pompiers et renforcer notre mod\xc3\xa8le de S\xc3\xa9curit\xc3\xa9 Civile. Rendez-vous ce mois-ci \xc3\xa0 l\'Assembl\xc3\xa9e Nationale pour l\'examen du texte. https://t.co/0305Wil94i",False,2021-05-08,famatras,M,0,0,True\r\n1,592816,"Je me r\xc3\xa9jouis de l\xe2\x80\x99annonce du @gouvernementFR ce matin qui maintient ses positions sur l\xe2\x80\x99\xc3\xa9tiquetage du #miel. \n<U+0001F41D>\nC\xe2\x80\x99est \xc3\xa9galement ce que je propose par un amendement sur la future #PPLegalim qui sera examin\xc3\xa9e en commission @AN_AfEco la semaine prochaine \xc3\xa0 l\xe2\x80\x99@AssembleeNat https://t.co/RAK1YLoxwY",False,2019-11-22,SandrineLeFeur,F,1,1,True\r\n2,11907,"Une fois gu\xc3\xa9ris, les enfants hospitalis\xc3\xa9s au @CH_Versailles peuvent embarquer

In [ ]:
df = pd.read_csv("df_label_sentiment.csv")
df. rename(columns = {'test':'Label == sentiment'}, inplace = True)

,Unnamed: 0,X,text,possibly_sensitive,created_at,twitter.x,sexe,Label,sentiment,Label == sentiment
0,0,248552,"Article de Var Matin, ce jour, sur la proposit...",False,2021-05-08,famatras,M,0,0,True
1,1,592816,Je me réjouis de l’annonce du @gouvernementFR ...,False,2019-11-22,SandrineLeFeur,F,1,1,True
2,2,11907,"Une fois guéris, les enfants hospitalisés au @...",False,2019-08-09,AdrienTaquet,M,1,1,True
3,3,74888,« Progressivement nous devons avoir une foncti...,False,2018-03-21,auroreberge,F,0,1,False
4,4,253273,"Réaction commune, avec ma collègue députée Sab...",False,2018-02-22,fabiennecolboc,F,0,0,True
...,...,...,...,...,...,...,...,...,...,...
995,995,81617,Le président de l'ONG #ASER qui lutte contre l...,False,2019-10-02,lachaudb,M,-1,-1,True
996,996,407056,Masques fournis à tous les élèves et étudiants...,False,2020-08-23,DeputeCausse,M,0,0,True
997,997,224729,Le nouveau monde macronien est celui de la dég...,False,2018-09-06,ECiotti,M,-1,-1,True
998,998,231496,"""Depuis le 2 Mai, l'Elysée sait que M. Benalla...",False,2018-07-23,ericcoquerel,M,-1,-1,True


In [ ]:
# Use TweetTokenizer

tok = TweetTokenizer()
tok.tokenize("I have a new GPU!".lower())

['i', 'have', 'a', 'new', 'gpu', '!']

In [ ]:
# Define useful functions
def tokenize_pad_numericalize(entry, vocab_stoi, max_length=20):
  text = [ vocab_stoi[token] if token in vocab_stoi else vocab_stoi['<unk>'] for token in tok.tokenize(entry.lower())]
  padded_text = None
  if len(text) < max_length:   padded_text = text + [ vocab_stoi['<pad>'] for i in range(len(text), max_length) ] 
  elif len(text) > max_length: padded_text = text[:max_length]
  else:                        padded_text = text
  return padded_text

def tokenize_all(entries, vocab_stoi):
  res = {}
  res['text'] = [tokenize_pad_numericalize(entry, vocab_stoi, max_length=200) for entry in entries['text']]
  res['label'] = entries['label']
  return res

In [ ]:
train, test = train_test_split(df, test_size=0.2)

## <font color="chillipepper">Pytorch Model</font>

Our goal is to define a deep learning model, train it, make prediction with it and evaluate it. 

With pytorch this means doing the three following "scripts" : 
1. Defining the model 
2. Implementing the prediction 
3. Implementing the training loop 
    - Defining a loss
    - Defining an optimizer
    - Loop :
        - forward pass 
        - backward pass
        - applying optimization update rule
4. Evaluating the model / playing with it 
    - You can use the training criteria (loss) as your evaluation score
    - You can use another score : accuracy, F1 , ...

In [ ]:
# defining the model 
class MinimalExampleModel(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(MinimalExampleModel, self).__init__()
        # define a first linear layer with an input dimension (D_in) and a Hidden dimension (H)
        # linear layer documentation: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
        self.linear1 = torch.nn.Linear(D_in, H, bias=True)
        # define the final linear layer, often named classification layer as the output dimension (D_out) == the number of target classes
        self.linear2 = torch.nn.Linear(H, D_out, bias=True)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        # first apply the linear1 layer on the vector and encapsulate it with an activation function for non linearization. This creates a dense layer.
        # relu documentation: https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#torch.nn.ReLU 
        h_relu = torch.relu(self.linear1(x))
        y_pred = self.linear2(h_relu)
        return y_pred

In [ ]:
# instanciating the model with
# N:      batch size
# D_in:   input dimension
# H:      hidden dimension
# D_out:  output dimension
N, D_in, H, D_out = 2, 10, 10, 2

# Construct our model by instantiating the class defined above 
# Note: all the parameters are initialized here 
model = MinimalExampleModel(D_in, H, D_out)
# You can look up into the model 
model

MinimalExampleModel(
  (linear1): Linear(in_features=10, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
# let's use torchinfo to have a better insight of the model infos: https://github.com/TylerYep/torchinfo
from torchinfo import summary
dummy_input_size = (1, 10) # (batch_size, D_in)
summary(model, (N, D_in))

Layer (type:depth-idx)                   Output Shape              Param #
MinimalExampleModel                      --                        --
├─Linear: 1-1                            [2, 10]                   110
├─Linear: 1-2                            [2, 2]                    22
Total params: 132
Trainable params: 132
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [ ]:
# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [ ]:
if torch.cuda.is_available():
  device = 'cuda'
  print('DEVICE = ', colored(torch.cuda.get_device_name(0), "green" ) )
else:
  device = 'cpu'
  print('DEVICE = ', colored('CPU', "blue"))
model = model.to(device)

#model = MyModel(D_in, H, D_out)
# forward pass / predict x 
y_pred = model(x.to(device)) # almost equivalent to model.forward(x)
# y_pred
y_pred

DEVICE =  CPU


tensor([[-0.4117, -0.1531],
        [-0.2486, -0.3666]], grad_fn=<AddmmBackward0>)

In [ ]:
# Create the model to classify tweets
class TweetModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, pretrained_vectors=None):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TweetModel, self).__init__()
        # apply the pretrained embeddings to transform our token indices, into vectors
        self.ebd = torch.nn.Embedding.from_pretrained(pretrained_vectors, freeze=True)
        self.hidden_linear_layer = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.classification_layer = torch.nn.Linear(hidden_dim, output_dim, bias=True)
        # softmax layer to compute class probabilities
        # https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html?highlight=softmax#torch.nn.Softmax
        self.softmax = nn.Softmax(dim=1)

        # define the dropout strategy (here, 20% (0.2) of the vector is ignored to prevent overfitting)
        # we don't use it here but it's a good thing to keep in mind
        # self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        # apply the pretrained embeddings
        x  = self.ebd(x)
        x  = x.mean(1)
        h  = torch.relu(self.hidden_linear_layer( x ))
        # h  = self.dropout(h)
        h  = self.classification_layer(h)
        logits = self.softmax(h)
        return logits